In [1]:
import sys
import os
from pathlib import Path  # makes dealing with paths in Wnidows easier - but takes only linux-style slashes
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import SimpleITK as sitk  # see doc at     https://itk.org/SimpleITKDoxygen/html/namespaceitk_1_1simple.html
import callElastix
import callTransformix

In [8]:

os.chdir(Path('S:/MR_datasets_local/Sub_3-2018-09-11/'))  # 'S:/MR_datasets_local/Sub_3-2018-09-11/'        # "S:/MR_datasets_local/Sub_7-2019-07-05/"        # "C:/Users/bzfmuham/OneDrive/Knee-Kinematics/elastix 4.9.0/elastix-4.9.0-example_2/exampleinput/"

I_f_name = 'R1_t1-minus-t2.nii'
I_m_name = 'R3_t1-minus-t2.nii'
I_f_mask_name = 'R1_wholeLegMask.labels.nii'
I_m_rigidityCoeffIm_name = 'R3_boneMask.labels.binary.nii'

# check if the "image-to-world" affine tranforms are the same for all involved volumes & masks
if np.array_equiv(nib.load(I_f_name).affine, nib.load(I_m_name).affine) is False \
or np.array_equiv(nib.load(I_f_name).affine, nib.load(I_f_mask_name).affine) is False \
or np.array_equiv(nib.load(I_f_name).affine, nib.load(I_m_rigidityCoeffIm_name).affine) is False :
    print('ERROR: "voxel-to-world" affine tranforms are not the same for all involved volumes & masks!')
    exit(-1)

In [ ]:
callElastix.callElastix(I_f_name = I_f_name,                # flexed, '72_Echo1.nii' 'R1_t1-minus-t2.nii' 'DataMergeR4_echo1.nii'
            I_m_name = I_m_name,                # extended, '70_Echo1.nii' 'R3_t1-minus-t2.nii' 'DataMergeR3_echo1.nii'
            I_f_mask_name = I_f_mask_name,       # 'wholeLeg_segmented_72.labels.nii' 'DataMergeR4_wholeLeg_segmented.labels'
            I_m_rigidityCoeffIm_name = I_m_rigidityCoeffIm_name,
            reg_type='RIGID',      # 'RIGID' - 'NON_RIGID' - 'PRE_ALLIGNED_NON_RIGID'
            n_itr_rigid= 1,                         # for all resolutions
            n_itr_nonRigid= 1,                      # 250 -> 2000 (good: 500) -  for all resolutions
            n_res= 1,                               # default: 4     (used for all registrations)
            I_deformed_name = 'deformed_im - rigid - without landmarks.nii'              #'deformed_im - rigCoefIm preAligned - itr = 2000.nii'
            )

## Transformix ###
# im_to_deform = sitk.ReadImage('rigidity_coeff_im_70.nii')
# # trnsfrmx_pMap = elstx_transform_pMap[0]
# trnsfrmx_pMap = sitk.ReadParameterFile('TransformParameters.0.txt')
# trnsfrmx_pMap['FinalBSplineInterpolationOrder'] = [
#     '0']  # A MUST when deforming a "binary" image (i.e. segmentation) (see doc)
# trnsfrmx_pMap['DefaultPixelValue'] = [
#     '0']  # sets pixel values outside the moving image grid (at interpolation) -> set it to <= the min in your dataset (i.e. bgd)
# trnsfrmx_deformed_Im = callTransformix.callTransformix(im_to_deform, trnsfrmx_pMap)
# sitk.WriteImage(trnsfrmx_deformed_Im, 'trnsfrmx_deformed_Im.nii')


### DSC
# LabelOverlapMeasuresImageFilter = sitk.LabelOverlapMeasuresImageFilter()


# ############# Im view (sitk -> calls ImageJ)  #############
#
# # Object oriented interface:
# image_viewer = sitk.ImageViewer()       # should be able to find ImageJ
# image_viewer.SetTitle('I_f')
# image_viewer.Execute(I_f)
# # image_viewer.SetTitle('I_m')
# # image_viewer.Execute(I_m)
# image_viewer.SetTitle('result im')
# image_viewer.Execute(resultImage)
# # image_viewer.SetTitle('deformation field')        # ImageJ can't view a 3d vector field
# # image_viewer.Execute(deformation_fld)